In [1]:
from tkinter import *
from PIL import Image, ImageTk
import os
from math import floor
path = os.path.join(os.curdir, 'images')

In [2]:
ims_path = os.listdir(path)
samp = os.path.join(path, 'samples')
os.mkdir(samp)
nothing = os.path.join(path, 'nothing')
os.mkdir(nothing)
print(samp, nothing)
ims_path.remove('nothing')
ims_path.remove('samples')

/home/boris-zverkov/images/samples /home/boris-zverkov/images/nothing


In [3]:
class Searching_Lines(Tk):
    dots = []
    img_number = 0
    x = []
    y = []
    R = []
    
    def __init__(self, filename):
        Tk.__init__(self)
        self.width = int(self.winfo_screenwidth() * 0.8)
        self.height = int(self.winfo_screenheight() * 0.8)
        
        self.ratio = float(self.width) / self.height
        self.title('Find tennis ball')
        
        self.photos = [os.path.join(path, i) for i in ims_path]
        self.result_file = open(filename, 'a')
        
        self.canvas = Canvas(self, width=self.width, height=self.height, bg="white")
        self.canvas.pack(side="top", fill="both", expand=True)
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.bind("1", self.check)
        self.bind("<Escape>", self.back)
        self.bind("<Tab>", self.next)
        
        self.draw_image()

    def draw_image(self):
        self.dots.clear()
        self.R.clear()
        self.x.clear()
        self.y.clear()
        self.image = Image.open(self.photos[self.img_number])
        w,h = self.image.size
        ratio = float(w)/h
        
        if ratio > self.ratio:
            self.coef = float(w) / self.width
            size = (self.width, int(floor(h / self.coef)))

        if ratio < self.ratio:
            self.coef = float(h) / self.height
            size = (int(floor(w / self.coef)), self.height)

        if ratio == self.ratio:
            self.coef = float(im_width / self.width)
            size = (self.width, self.height)
        
        self.resized = self.image.resize(size, Image.ANTIALIAS)
        self.tk_image = ImageTk.PhotoImage(self.resized)
        self.canvas.create_image(0, 0, anchor="nw", image=self.tk_image)
        
        
    def check(self, event):
        for i in range(len(self.dots) // 3):
            x12 = self.dots[i * 3][0] - self.dots[i * 3 + 1][0]
            x23 = self.dots[i * 3 + 1][0] - self.dots[i * 3 +2][0]
            x31 = self.dots[i * 3 + 2][0] - self.dots[i * 3][0]
            y12 = self.dots[i * 3][1] - self.dots[i * 3 + 1][1]
            y23 = self.dots[i * 3 +1][1] - self.dots[i * 3 + 2][1]
            y31 = self.dots[i * 3 + 2][1] - self.dots[i * 3][1]
            z1 = self.dots[i * 3][0] ** 2 + self.dots[i * 3][1] ** 2
            z2 = self.dots[i * 3 + 1][0] ** 2 + self.dots[i * 3 + 1][1] ** 2
            z3 = self.dots[i * 3 + 2][0] ** 2 + self.dots[i * 3 + 2][1] ** 2
            zx = y12 * z3 + y23 * z1 + y31 * z2
            zy = x12 * z3 + x23 * z1 + x31 * z2
            z = x12 * y31 - y12 * x31
            xx = (-1) * zx / (2 * z)
            self.x.append(xx)
            yy = zy / (2 * z)
            self.y.append(yy)
            a = (x12 ** 2 + y12 **2) ** 0.5
            b = (x23 ** 2 + y23 **2) ** 0.5
            c = (x31 ** 2 + y31 **2) ** 0.5
            p = (a + b + c) / 2
            S = (p * (p - a) * (p - b) * (p - c)) ** 0.5
            r = a * b * c / (4 * S)
            self.R.append(r)
            self.canvas.create_oval(xx - r, yy - r,
                                   xx + r, yy + r,
                                    fill='Red', outline="Red")
    
    def back(self, event):
        self.canvas.delete()
        self.dots.clear()
        self.R.clear()
        self.x.clear()
        self.y.clear()
        self.draw_image()
        
    def on_button_press(self, event):
            self.dots.append([event.x, event.y])
        
            
    def f(self, x):
        return self.coef*x
    
    def next(self, event):
        cur = os.path.join(path, ims_path[self.img_number])
        if len(self.dots) != 0:
            self.result_file.write(ims_path[self.img_number])
            self.result_file.write('\n')
            self.result_file.write(str(len(self.R)) + '\n')
            for i in range(len(self.R)):
                self.result_file.write(str(self.f(self.x[i])) + '\n' + str(self.f(self.y[i])) 
                                       + '\n' + str(self.f(self.R[i])) + '\n')
            dest = os.path.join(samp, ims_path[self.img_number])
            os.rename(cur, dest)
        else:
            dest = os.path.join(nothing, ims_path[self.img_number])
            os.rename(cur, dest)
        self.img_number += 1
        if self.img_number >= len(self.photos):
            self.result_file.close()
            self.destroy()
        else:
            self.draw_image()

        
if __name__ == '__main__':
    app = Searching_Lines(os.path.join(samp, 'dataset.txt'))
    app.mainloop()

1
187.97516394060452 407.64473882565767 142.94987491561284
1
841.8261290322581 716.2841935483871 169.3370261237665
1
862.0443037974684 722.373417721519 109.0570151015593
1
639.0271981165769 475.50318884523716 349.2676256454883
1
364.3979591836735 608.0510204081633 77.9654407682388
1
720.3990303069095 487.91037539603803 277.1955698609882
1
133.77513499614298 684.1760092568784 55.189603113202416
1
623.8563516730591 456.83214852532734 321.0557373868268
1
572.1630541871921 414.6949843260188 180.15468289504037
1
437.570897986134 429.971122895345 339.20398313094825
1
389.63208659483183 469.9721448924115 225.20945804224402
1
327.0 320.77868852459017 39.53873131312023
1
290.97010591048854 627.4496071062522 48.0796987341555
1
261.34781746905696 634.5278817777386 93.74305017680574
1
377.7708288940976 380.7083954826337 252.84115612583253
1
345.2579674357589 476.80229956015125 146.4531643687768
1
436.90909090909093 813.9090909090909 37.115522136581525
1
431.0104342921602 735.0132543711223 94.57267

0
1
-156.07151300236407 410.3486997635934 705.0500931124162
1
949.738432092409 553.4751068926149 141.6212621484967
1
808.9154071184788 550.7775475377864 57.8598281428998
1
576.6592227776266 452.4544532316469 436.00240623147334
4
210.408475925189 729.0319007825972 76.70288443684802
225.73401503516288 575.3700185918681 74.5156575324611
229.3428273706148 433.4033518582841 71.59727607933094
235.8709073900842 296.1421889616464 70.84068495573364
1
294.24570081667883 162.90150056437156 83.85598604340296
1
231.71932967481519 280.24742618712867 199.38332165854052
1
435.6074939952838 437.39859826123114 393.95738598604686
1
803.6434977578475 391.2130044843049 70.42982894744358
1
786.5078777751253 643.8060396275961 95.70574644599802
1
425.61001024940214 709.2707550392894 90.90125321599321
1
431.48684500624677 417.4019622253252 343.5209531156005
1
296.660344067049 511.9704455227172 237.36874720677804
1
354.9574216940128 479.45608687018745 238.2233772415443
1
945.2318326031123 460.0071007705091 174.